<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/EIA/weeklyRefineryUtilizationAndChanges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
#import os

In [2]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [3]:
url = 'https://api.eia.gov/v2/petroleum/pnp/wiup/data/?api_key='+ api_key + '&data[0]=value&frequency=weekly&start=2014-01-01&sort[0][column]=period&sort[0][direction]=asc&facets[series][]='

In [4]:
#https://www.eia.gov/opendata/browser/petroleum/pnp/wiup?frequency=weekly&data=value;&sortColumn=period;&sortDirection=desc;
response = requests.get(url + 'WPULEUS3')

In [11]:
df = pd.DataFrame(response.json().get('response').get('data'))[['period','value']]
df.columns = ['Period', 'value']
df['Period'] = df['Period'].apply(pd.to_datetime)
df['value'] = pd.to_numeric(df['value'], errors='coerce') # errors='coerce' will replace invalid values with NaN
#df.value = (df.value/1).round(1) #.astype('Int64')
df['year'] = df['Period'].dt.year
df['week'] = df['Period'].dt.isocalendar().week
df['change'] = df['value'] - df['value'].shift(1)
df.tail(3)

,Period,value,year,week,change
558,2024-09-13,92.1,2024,37,-0.7
559,2024-09-20,90.9,2024,38,-1.2
560,2024-09-27,87.6,2024,39,-3.3


In [12]:
ty = df.year.iloc[-1]
ly = ty-1

In [13]:
dfA = df.pivot(index='week', columns='year', values='value').round(1) #.astype('Int64') #.reset_index(drop='true')
dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [14]:
dfAA = pd.DataFrame(dfA.iloc[:, -6:-1].mean(axis=1))
dfAA.columns = ['5yAvg']
dfAA['5yMin'] = dfA.iloc[:, -6:-1].min(axis=1)
dfAA['5yMaxA'] = dfA.iloc[:, -6:-1].max(axis=1)
dfAA['5yMax'] = dfAA['5yMaxA']-dfAA['5yMin']
dfAA['10yAvg'] = dfA.iloc[:, -11:-1].mean(axis=1)
dfAA['10yMin'] = dfA.iloc[:, -11:-1].min(axis=1)
dfAA['10yMax'] = dfA.iloc[:, -11:-1].max(axis=1)
dfAA['week'] = dfA['week']

In [15]:
dfA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfA['week'].astype(str)+'5',format='%Y%W%w')
dfAA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAA['week'].astype(str)+'5',format='%Y%W%w')

In [16]:
dfAA.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,10yAvg,10yMin,10yMax,week,weekDate
0,88.72,82.0,96.1,14.1,90.99,82.0,96.1,1,2024-01-05
1,88.54,82.5,94.6,12.1,89.80,82.5,94.6,2,2024-01-12
2,87.78,81.7,92.9,11.2,87.75,81.7,92.9,3,2024-01-19


In [20]:
# Create the area chart
fig = px.area(dfAA, x=dfAA.weekDate, y=['5yMin', '5yMax'], range_y=[50, 100], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

fig.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

fig.add_scatter(x=dfAA.weekDate, y=dfAA['5yAvg'], mode='lines', name='5yAvg', line=dict(color='darkgrey', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    title={
        'text': '<b>Refinery Utilization</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Refinery Utilization (%)'

)


fig.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


fig.add_scatter(x = [dfA['weekDate'][dfA[ty].isna().idxmax()-1]], y = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='top right') # top, bottom, middle


fig.show()

In [21]:
dfAc = df.pivot(index='week', columns='year', values='change').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAc = dfAc.drop(dfAc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfAc.tail(3)

year,week,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
49,50,-1.9,-1.2,1.0,0.7,0.3,0.0,-0.8,-0.2,-1.3,2.2,NaN
50,51,0.0,-0.6,-0.5,1.6,-0.3,2.7,-1.1,0.1,1.1,0.9,NaN
51,52,0.9,1.3,1.0,1.0,2.1,1.2,1.4,0.1,-12.4,0.2,NaN


In [22]:
dfAAc = pd.DataFrame(dfAc.iloc[:, -6:-1].mean(axis=1))
dfAAc.columns = ['5yAvg']
dfAAc['5yMin'] = dfAc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAAc['5yMaxA'] = dfAc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAAc['5yMax'] = dfAAc['5yMaxA']-dfAAc['5yMin']
dfAAc['week'] = dfAc['week']
dfAAc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,0.36,-1.5,4.5,6.0,1
1,-0.18,-1.5,1.2,2.7,2
2,-0.76,-1.7,0.8,2.5,3


In [23]:
dfAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAc['week'].astype(str)+'5',format='%Y%W%w')
dfAAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAc['week'].astype(str)+'5',format='%Y%W%w')

In [24]:
categories = dfAAc['weekDate']
lower_bounds = dfAAc['5yMin']
upper_bounds = dfAAc['5yMaxA']

In [27]:
fig = go.Figure()

for i in range(len(categories)):
    fig.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

fig.add_scatter(x=dfAAc.weekDate, y=dfAAc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
fig.add_scatter(x=dfAc.weekDate, y=dfAc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfAc.weekDate, y=dfAc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

fig.add_scatter(x = [dfAc['weekDate'][dfAc[ty].isna().idxmax()-1]], y = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom center') # top left, bottom right, middle left, bottom righ, top center, auto center

fig.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Refinery Utilization Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='Refinery Utilization Weekly Change (%)')

# Add source link as an annotation
fig.add_annotation(
    text="Source: EIA     https://www.eia.gov/petroleum/supply/weekly",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

fig.show()